# Voorbeeld van omzetting door Veg2Hab

Dit notebook doorloopt alle stappen die tot nu toe geimplementeerd zijn voor Veg2Hab. Het is een voorbeeld van hoe de omzetting van vegetatie naar habitattypekaart kan verlopen.

Eerst worden alle benodigde resources geinstantieerd (was-wordt lijst, definitietabel, fgr-kaart). Hierna wordt de omzetting stap voor stap uitgevoerd.

In [1]:
from pathlib import Path

import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_was_wordt_lijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
import pandas as pd
from veg2hab.fgr import FGR

pd.set_option('display.max_columns', 100)

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Was-Wordt lijst opschonen en inladen

VvN en SBB codes worden gecheckt op validiteit bij:
1. het opschonen van een was-wordt lijst
2. bij het inlezen van een opgeschoonde was-wordt lijst

In [2]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_was_wordt_lijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)
wwl.df.head(3)

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide


,VvN,SBB
0,14rg10,14-l
1,None,7-a
2,None,8b-b


### Definitietabel opschonen en inladen

VvN- en SBB-codes worden gecheckt op validiteit bij:
1. het opschonen van een definitietabel
2. bij het inlezen van een opgeschoonde definitietabel

In [3]:
path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_mitsjson = Path("../data/mitsjson.csv")
path_in_mozaiekjson = Path("../data/mozaiekjson.csv")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_mitsjson, path_in_mozaiekjson, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)
dt.df.head()

Alle SBB codes zijn valide
Alle VvN codes zijn valide


,DT regel,Habitattype,Kwaliteit,SBB,VvN,mits,mozaiek,mitsjson,mozaiekjson,Criteria,Mozaiekregel
0,2,H1110_A,Kwaliteit.GOED,50a,None,mits in de slikkige en fijnzandige delen van F...,<NA>,"{ ""type"": ""EnCriteria"", ""sub_criteria""...","{ ""type"" : ""GeenMozaiekregel"" }",((FGR is Getijdengebied of FGR is Noordzee) en...,mozaiek_threshold=90
1,3,H1110_A,Kwaliteit.GOED,50b,None,mits in de slikkige en fijnzandige delen van F...,<NA>,"{ ""type"": ""EnCriteria"", ""sub_criteria""...","{ ""type"" : ""GeenMozaiekregel"" }",((FGR is Getijdengebied of FGR is Noordzee) en...,mozaiek_threshold=90
2,4,H1110_A,Kwaliteit.GOED,50c,None,mits in de slikkige en fijnzandige delen van F...,<NA>,"{ ""type"": ""EnCriteria"", ""sub_criteria""...","{ ""type"" : ""GeenMozaiekregel"" }",((FGR is Getijdengebied of FGR is Noordzee) en...,mozaiek_threshold=90
3,5,H1110_B,Kwaliteit.GOED,50a,None,mits in de grofzandige delen van FGR Getijdeng...,<NA>,"{ ""type"": ""EnCriteria"", ""sub_criteria""...","{ ""type"" : ""GeenMozaiekregel"" }",((FGR is Getijdengebied of FGR is Noordzee) en...,mozaiek_threshold=90
4,6,H1110_B,Kwaliteit.GOED,50b,None,mits in de grofzandige delen van FGR Getijdeng...,<NA>,"{ ""type"": ""EnCriteria"", ""sub_criteria""...","{ ""type"" : ""GeenMozaiekregel"" }",((FGR is Getijdengebied of FGR is Noordzee) en...,mozaiek_threshold=90


In [4]:
dt.df.Mozaiekregel.iloc[-6]

StandaardMozaiekregel(mozaiek_threshold=90, habtype='H91E0_C', alleen_zelfstandig=True, alleen_goede_kwaliteit=False, keys=[('H91E0_C', True, <Kwaliteit.GOED: 'Goed'>), ('H91E0_C', True, <Kwaliteit.MATIG: 'Matig'>)])

### FGR inladen

Bij het inladen worden de FGR typen omgezet naar de FGRType Enum, en hierdoor vind ook validatie plaats.

In [5]:
fgr = FGR(Path("../data/bronbestanden/FGR.json"))
fgr.gdf.head(3)

,fgr,geometry
0,FGRType.DU,"POLYGON ((105147.000 526863.188, 105092.000 52..."
1,FGRType.DU,"POLYGON ((111314.000 552409.000, 111289.008 55..."
2,FGRType.DU,"POLYGON ((119244.333 577690.288, 119230.924 57..."


## Inladen van een kartering vanaf een shapefile

In [6]:
shp_path = Path("../testing/vegetatiekarteringen/GR/SGL Hunzedal en Leekstermeer2021/2021 Vegetatiekartering Leekstermeer2021/GIS bestanden Onlanden 2021/Vegetatiekartering_Leekstermeer2021.shp")
Kartering.from_shapefile(shp_path, "elmid", vegtype_col_format="single", sbb_of_vvn="sbb", SBB_col="SBBTYPE", split_char="+").gdf

,elmid,geometry,SBBTYPE,SBBTYPE1,SBBTYPE2,SBBTYPE3,Datum,Opmerking,Opp,_LokVrtNar,perc_0,perc_1,perc_2,VegTypeInfo
0,144,"POLYGON ((225536.382 578550.118, 225536.613 57...",08C-a+16-a,8c-a,16-a,None,None,None,1692.139818,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['8c-a'], VvN: []), (50.0%, SBB:..."
1,143,"POLYGON ((225518.346 578389.608, 225514.596 57...",08-a+08C2c,8-a,8c2c,None,None,None,2974.448552,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['8-a'], VvN: []), (50.0%, SBB: ..."
2,73,"POLYGON ((225398.991 578372.192, 225399.792 57...",16-a,16-a,None,None,None,None,4582.870987,Lokale typologie is primair vertaald naar sbb,100.0,0.0,0.0,"[(100.0%, SBB: ['16-a'], VvN: [])]"
3,62,"POLYGON ((225338.120 578506.385, 225330.908 57...",12B1d+50A,12b1d,50a,None,None,None,2843.780077,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['12b1d'], VvN: []), (50.0%, SBB..."
4,111,"POLYGON ((225731.271 578748.007, 225730.075 57...",50A+08B3d,50a,8b3d,None,None,None,1911.539469,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['50a'], VvN: []), (50.0%, SBB: ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,193,"POLYGON ((225335.414 578905.115, 225334.448 57...",12B1d,12b1d,None,None,None,None,442.032701,Lokale typologie is primair vertaald naar sbb,100.0,0.0,0.0,"[(100.0%, SBB: ['12b1d'], VvN: [])]"
267,220,"POLYGON ((225121.987 578757.930, 225125.003 57...",08-k,8-k,None,None,None,None,361.533352,Lokale typologie is primair vertaald naar sbb,100.0,0.0,0.0,"[(100.0%, SBB: ['8-k'], VvN: [])]"
268,58,"POLYGON ((225297.768 578230.760, 225292.476 57...",08B3a+08C-f,8b3a,8c-f,None,None,None,722.439797,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['8b3a'], VvN: []), (50.0%, SBB:..."
269,249,"POLYGON ((224515.275 578618.372, 224510.014 57...",39A1c+08B3a,39a1c,8b3a,None,None,None,713.229564,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['39a1c'], VvN: []), (50.0%, SBB..."


## Inladen en omzetten van een access kartering

- Voor ieder vlak in de kartering wordt de (complexe) SBB code opgezocht.
- Deze SBB-codes worden opgeschoond en gevalideerd.
- De SBB codes worden toegevoegd aan de dataframe van de kartering als VegTypeInfo instance.
  
De resulterende dataframe heeft voor elke regel in KarteringVegetatietype.csv een VegTypeInfo instance in de rij van de betreffende shape.

In [7]:
shape_path = Path("../data/notebook_data/Rottige_Meenthe_Brandemeer_2013/vlakken.shp")
csvs_path = Path("../data/notebook_data/Rottige_Meenthe_Brandemeer_2013/864_RottigeMeenthe2013/")
shape_elm_id_column = "ElmID"

access_kartering = Kartering.from_access_db(shape_path, shape_elm_id_column, csvs_path)

access_kartering.gdf.head(3)


,ElmID,Opmerking,Datum,geometry,Opp,_LokVrtNar,intern_id,Locatie,VegTypeInfo
0,28049,None,19072013,"POLYGON ((189433.818 538314.302, 189439.215 53...",3761.736281,Lokale typologie is primair vertaald naar SBB,1809,1809,"[(90%, SBB: ['9a2a'], VvN: []), (10%, SBB: ['9..."
1,15034,None,10092013,"POLYGON ((190568.500 539171.122, 190538.023 53...",10442.658431,Lokale typologie is primair vertaald naar SBB,815,815,"[(100%, SBB: ['50a'], VvN: [])]"
2,13503,None,14102013,"POLYGON ((189538.104 539409.332, 189532.399 53...",2174.098680,Lokale typologie is primair vertaald naar SBB,650,650,"[(90%, SBB: ['9a2a'], VvN: []), (10%, SBB: ['9..."


#### Opzoeken en toevoegen VvN aan kartering

In [8]:
access_kartering.apply_wwl(wwl)
# Voorbeeld van 3 vegtypeinfos in 1 geometry
access_kartering.gdf["VegTypeInfo"].iloc[3]

[VegTypeInfo(percentage=80, SBB=[SBB(klasse='9', verbond='b', associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='a')], VvN=[]),
 VegTypeInfo(percentage=10, SBB=[SBB(klasse='9', verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='g')], VvN=[VvN(klasse='9', orde=None, verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='3')]),
 VegTypeInfo(percentage=10, SBB=[SBB(klasse='9', verbond='a', associatie='2', subassociatie='b', derivaatgemeenschap=None, rompgemeenschap=None)], VvN=[VvN(klasse='9', orde='a', verbond='a', associatie='2', subassociatie='b', derivaatgemeenschap=None, rompgemeenschap=None)])]

#### Vinden van de mogelijke habitattypen voor een VegTypeInfo

Voor elke VegTypeInfo worden de mogelijke habitattypen opgezocht in de definitietabel. Deze worden vervolgens in een HabitatVoorstel gezet, samen met de code waarop de match is gemaakt, de regel in de definitietabel die is gebruikt en het niveau van de match.

In [9]:
access_kartering.apply_deftabel(dt)
access_kartering.gdf["HabitatVoorstel"]

0       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1       [[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...
2       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
3       [[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...
4       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
                              ...                        
1944    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1945    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1946    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1947    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1948    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
Name: HabitatVoorstel, Length: 1949, dtype: object

In [10]:
# Voorbeeld met voor 2 VegTypeInfo's waarbij de eerste twee mogelijke voorstellen heeft
access_kartering.gdf["HabitatVoorstel"][45]

[[HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie='a', derivaatgemeenschap=None, rompgemeenschap=None), vegtype_in_dt=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), habtype='H2180_B', kwaliteit=<Kwaliteit.GOED: 'Goed'>, idx_in_dt=178, mits=FGRCriterium(fgrtype=<FGRType.DU: 'Duinen'>), mozaiek=GeenMozaiekregel(mozaiek_threshold=90), match_level=6),
  HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie='a', derivaatgemeenschap=None, rompgemeenschap=None), vegtype_in_dt=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), habtype='H91D0', kwaliteit=<Kwaliteit.GOED: 'Goed'>, idx_in_dt=650, mits=EnCriteria(sub_criteria=[NietCriterium(sub_criterium=FGRCriterium(fgrtype=<FGRType.DU: 'Duinen'>)), PlaceholderCriterium()]), mozaiek=Geen

In [11]:
# voor dev

In [20]:
access_kartering.bepaal_habtitatkeuzes(fgr)

/home/jordydelange/git/veg2hab/veg2hab/mozaiek.py:185: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:28992

  buffered_boundary, only_needed_cols, how="union", keep_geom_type=True


Iteratie 0: van 3499 naar 108 keuzes nog te bepalen
Iteratie 1: van 108 naar 1 keuzes nog te bepalen
Iteratie 2: van 1 naar 0 keuzes nog te bepalen


In [24]:
for x in access_kartering.gdf[access_kartering.gdf["ElmID"] == 7045]["HabitatVoorstel"].iloc[0]:
    print(x)

[HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='9', orde='a', verbond='a', associatie='2', subassociatie='b', derivaatgemeenschap=None, rompgemeenschap=None), vegtype_in_dt=VvN(klasse='9', orde='a', verbond='a', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), habtype='H7140_B', kwaliteit=<Kwaliteit.GOED: 'Goed'>, idx_in_dt=566, mits=GeenCriterium(), mozaiek=GeenMozaiekregel(mozaiek_threshold=90), match_level=6)]
[HabitatVoorstel(onderbouwend_vegtype=SBB(klasse='9', verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap='c', rompgemeenschap=None), vegtype_in_dt=SBB(klasse='9', verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap='c', rompgemeenschap=None), habtype='H2190_C', kwaliteit=<Kwaliteit.MATIG: 'Matig'>, idx_in_dt=235, mits=GeenCriterium(), mozaiek=StandaardMozaiekregel(mozaiek_threshold=90, habtype='H2190_C', alleen_zelfstandig=True, alleen_goede_kwaliteit=False, keys=[('H2190_C', True, <Kwaliteit.GOED: 'G

In [16]:
# save access_kartering.gdf to tmp.shp
access_kartering.gdf[["finished_on_iteration", "geometry", "ElmID"]].to_file("tmp.shp")

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


In [18]:
g = access_kartering.gdf
g

,ElmID,Opmerking,Datum,geometry,Opp,_LokVrtNar,intern_id,Locatie,VegTypeInfo,HabitatVoorstel,HabitatKeuze,finished_on_iteration
0,28049,None,19072013,"POLYGON ((189433.818 538314.302, 189439.215 53...",3761.736281,Lokale typologie is primair vertaald naar SBB,1809,1809,"[(90%, SBB: ['9a2a'], VvN: ['9aa2a']), (10%, S...",[[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...,[HabitatKeuze(status=<KeuzeStatus.DUIDELIJK: 1...,1
1,15034,None,10092013,"POLYGON ((190568.500 539171.122, 190538.023 53...",10442.658431,Lokale typologie is primair vertaald naar SBB,815,815,"[(100%, SBB: ['50a'], VvN: [])]",[[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...,[HabitatKeuze(status=<KeuzeStatus.PLACEHOLDER_...,1
2,13503,None,14102013,"POLYGON ((189538.104 539409.332, 189532.399 53...",2174.098680,Lokale typologie is primair vertaald naar SBB,650,650,"[(90%, SBB: ['9a2a'], VvN: ['9aa2a']), (10%, S...",[[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...,[HabitatKeuze(status=<KeuzeStatus.DUIDELIJK: 1...,1
3,15075,None,12092013,"POLYGON ((190801.313 539561.740, 190819.617 53...",18878.050297,Lokale typologie is primair vertaald naar SBB,2515,2515,"[(80%, SBB: ['9b-a'], VvN: []), (10%, SBB: ['9...",[[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...,[HabitatKeuze(status=<KeuzeStatus.GEEN_KLOPPEN...,1
4,20502,None,02102013,"POLYGON ((190497.231 537724.945, 190505.062 53...",4950.611630,Lokale typologie is primair vertaald naar SBB,1316,1316,"[(58%, SBB: ['8b3c'], VvN: ['8bb4d']), (29%, S...",[[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...,[HabitatKeuze(status=<KeuzeStatus.VEGTYPEN_NIE...,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1944,25047,None,22082013,"POLYGON ((188748.713 538720.165, 188740.937 53...",929.112247,Lokale typologie is primair vertaald naar SBB,1564,1564,"[(90%, SBB: ['36a2'], VvN: ['36aa2a']), (10%, ...",[[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...,[HabitatKeuze(status=<KeuzeStatus.PLACEHOLDER_...,1
1945,25050,None,22082013,"POLYGON ((188717.598 538767.120, 188713.450 53...",900.019817,Lokale typologie is primair vertaald naar SBB,1567,1567,"[(90%, SBB: ['32-c'], VvN: ['32rg3']), (10%, S...",[[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...,[HabitatKeuze(status=<KeuzeStatus.VEGTYPEN_NIE...,1
1946,26013,None,28082013,"POLYGON ((189977.881 538379.974, 189975.341 53...",110.775294,Lokale typologie is primair vertaald naar SBB,2095,2095,"[(60%, SBB: ['8c1c'], VvN: ['8ba2']), (30%, SB...",[[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...,[HabitatKeuze(status=<KeuzeStatus.PLACEHOLDER_...,1
1947,26313,None,29082013,"POLYGON ((189897.152 537982.231, 189906.500 53...",2740.769980,Lokale typologie is primair vertaald naar SBB,2496,2496,"[(90%, SBB: ['5c1'], VvN: ['5bb1']), (10%, SBB...",[[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...,[HabitatKeuze(status=<KeuzeStatus.PLACEHOLDER_...,1


In [19]:
none_present_in_habitatkeuze = g.HabitatKeuze.apply(lambda keuzes: keuzes.count(None)) > 0
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 250)
pd.set_option('display.width', 250)
g[g["ElmID"]==7045][none_present_in_habitatkeuze]

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/geodataframe.py:1327: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


,ElmID,Opmerking,Datum,geometry,Opp,_LokVrtNar,intern_id,Locatie,VegTypeInfo,HabitatVoorstel,HabitatKeuze,finished_on_iteration


In [21]:
for sublist in g[none_present_in_habitatkeuze].iloc[2].HabitatVoorstel:
    for voorstel in sublist:
        print(voorstel)
    print("---------------------------------")

HabitatVoorstel(onderbouwend_vegtype=SBB(klasse='9', verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='i'), vegtype_in_dt=SBB(klasse='9', verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='i'), habtype='H7140_A', kwaliteit=<Kwaliteit.MATIG: 'Matig'>, idx_in_dt=565, mits=GeenCriterium(), mozaiek=StandaardMozaiekregel(mozaiek_threshold=90, habtype='H7140_A', alleen_zelfstandig=True, alleen_goede_kwaliteit=False, keys=[('H7140_A', True, <Kwaliteit.GOED: 'Goed'>), ('H7140_A', True, <Kwaliteit.MATIG: 'Matig'>)]), match_level=11)
HabitatVoorstel(onderbouwend_vegtype=SBB(klasse='9', verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='i'), vegtype_in_dt=SBB(klasse='9', verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='i'), habtype='H7140_B', kwaliteit=<Kwaliteit.MATIG: 'Matig'>, idx_in_dt=571, mits=GeenCriterium(), mozaiek=

### De mitsen checken en de HabitatVoorstellen omzetten naar HabitatKeuzes

In [11]:
access_kartering.check_mitsen(fgr)
print(access_kartering.gdf["HabitatKeuze"][5][1])
print(access_kartering.gdf["VegTypeInfo"][5][1])

KeyError: 'HabitatKeuze'

### De kartering formatten als een Habitattypenkartering

Dit pakt de HabitatVoorstellen uit en format het geheel als een Habitattypenkartering zoals beschreven in Gegevens Leverings Protocol (uitvraag bijlage 3a).

Voor nu checken we nog geen mitsen en mozaiek en wordt voor ieder VegTypeInfo het eerste HabitatVoorstel gebruikt.

In [ ]:
final_format = access_kartering.as_final_format()
final_format

,Area,Opm,Datum,geometry,_ChkNodig,_Samnvttng,_LokVrtNar,Habtype1,Perc1,Opp1,Kwal1,Opm1,VvN1,SBB1,_Status1,_Uitleg1,_VvNdftbl1,_SBBdftbl1,_VgTypInf1,_ChkNodig1,Habtype2,Perc2,Opp2,Kwal2,Opm2,VvN2,SBB2,_Status2,_Uitleg2,_VvNdftbl2,_SBBdftbl2,_VgTypInf2,_ChkNodig2,Habtype3,Perc3,Opp3,Kwal3,Opm3,VvN3,SBB3,_Status3,_Uitleg3,_VvNdftbl3,_SBBdftbl3,_VgTypInf3,_ChkNodig3,Habtype4,Perc4,Opp4,Kwal4,Opm4,VvN4,SBB4,_Status4,_Uitleg4,_VvNdftbl4,_SBBdftbl4,_VgTypInf4,_ChkNodig4,Habtype5,Perc5,Opp5,Kwal5,Opm5,VvN5,SBB5,_Status5,_Uitleg5,_VvNdftbl5,_SBBdftbl5,_VgTypInf5,_ChkNodig5
0,3761.736281,None,19072013,"POLYGON ((189433.818 538314.302, 189439.215 53...",False,100.0% H7140_B,Lokale typologie is primair vertaald naar SBB,H7140_B,90,338556,G,Er is een duidelijke keuze. Kloppende mits: ge...,9aa2a,None,KeuzeStatus.DUIDELIJK,Als alle regels gevolgd worden is er 1 duideli...,"['9aa2', 566]",None,"(90%, SBB: ['9a2a'], VvN: ['9aa2a'])",False,H7140_B,10,37617.4,G,Er is een duidelijke keuze. Kloppende mits: ge...,9aa2a,None,KeuzeStatus.DUIDELIJK,Als alle regels gevolgd worden is er 1 duideli...,"['9aa2', 566]",None,"(10%, SBB: ['9a2a'], VvN: ['9aa2a'])",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10442.658431,None,10092013,"POLYGON ((190568.500 539171.122, 190538.023 53...",True,100.0% HXXXX,Lokale typologie is primair vertaald naar SBB,HXXXX,100,1044265.8431475804,X,Er zijn mitsen met nog niet geimplementeerde c...,"[None, None, None, None, None, None, None, Non...","['50a', '50a', '50a', '50a', '50a', '50a', '50...",KeuzeStatus.PLACEHOLDER_CRITERIA,Er zijn placeholder criteria gevonden; deze ku...,"[None, None, None, None, None, None, None, Non...","[""['50a', 2]"", ""['50a', 5]"", ""['50a', 12]"", ""[...","(100%, SBB: ['50a'], VvN: [])",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2174.098680,None,14102013,"POLYGON ((189538.104 539409.332, 189532.399 53...",False,100.0% H7140_B,Lokale typologie is primair vertaald naar SBB,H7140_B,90,195669,G,Er is een duidelijke keuze. Kloppende mits: ge...,9aa2a,None,KeuzeStatus.DUIDELIJK,Als alle regels gevolgd worden is er 1 duideli...,"['9aa2', 566]",None,"(90%, SBB: ['9a2a'], VvN: ['9aa2a'])",False,H7140_B,10,21741,G,Er is een duidelijke keuze. Kloppende mits: ge...,9aa2a,None,KeuzeStatus.DUIDELIJK,Als alle regels gevolgd worden is er 1 duideli...,"['9aa2', 566]",None,"(10%, SBB: ['9a2a'], VvN: ['9aa2a'])",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18878.050297,None,12092013,"POLYGON ((190801.313 539561.740, 190819.617 53...",True,"80.0% H3160, 10.0% H7140_B, 10.0% HXXXX",Lokale typologie is primair vertaald naar SBB,H3160,80,1.51024e+06,G,Er is een duidelijke keuze. Kloppende mits: ge...,None,9b-a,KeuzeStatus.DUIDELIJK,Als alle regels gevolgd worden is er 1 duideli...,None,"['9b-a', 342]","(80%, SBB: ['9b-a'], VvN: [])",False,HXXXX,10,188781,X,Er zijn mitsen met nog niet geimplementeerde c...,9rg3,None,KeuzeStatus.PLACEHOLDER_CRITERIA,Er zijn placeholder criteria gevonden; deze ku...,"['9rg3', 230]",None,"(10%, SBB: ['9-g'], VvN: ['9rg3'])",True,H7140_B,10,188781,G,Er is een duidelijke keuze. Kloppende mits: ge...,9aa2b,None,KeuzeStatus.DUIDELIJK,Als alle regels gevolgd worden is er 1 duideli...,"['9aa2', 566]",None,"(10%, SBB: ['9a2b'], VvN: ['9aa2b'])",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4950.611630,None,02102013,"POLYGON ((190497.231 537724.945, 190505.062 53...",True,"61.0% H0000, 39.0% HXXXX",Lokale typologie is primair vertaald naar SBB,HXXXX,29,143568,X,Er zijn mitsen met nog niet geimplementeerde c...,8bd2,None,KeuzeS

In [ ]:
access_kartering.final_format_to_file(Path("../testing/veg2hab_output.shp"))

KeyboardInterrupt: 